In [44]:
#https://www.kaggle.com/code/akshaychavan123/taxi-trip-time-prediction/notebook
import pandas as pd
import seaborn as sns
import datetime

In [33]:
baza = pd.read_csv('taxi 400k.csv')

In [34]:
baza.head(4)

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."


In [35]:
baza.describe()

,TRIP_ID,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP
count,4.000000e+05,82253.000000,198364.000000,4.000000e+05,4.000000e+05
mean,1.376474e+18,23486.085663,30.422607,2.000035e+07,1.376474e+09
std,2.286724e+15,19692.943272,17.825948,2.098117e+02,2.286724e+06
min,1.372637e+18,2001.000000,1.000000,2.000000e+07,1.372637e+09
25%,1.374427e+18,5444.000000,15.000000,2.000017e+07,1.374427e+09
50%,1.376425e+18,17217.000000,27.000000,2.000035e+07,1.376425e+09
75%,1.378560e+18,39654.000000,49.000000,2.000052e+07,1.378560e+09
max,1.380280e+18,63882.000000,63.000000,2.000098e+07,1.380280e+09


In [36]:
#usuwanie kolumn
baza.drop(['CALL_TYPE', 'ORIGIN_CALL','ORIGIN_STAND','TAXI_ID'], axis=1, inplace=True)

In [37]:
len(baza)

400000

In [38]:
def usun_wiersze_z_brakami(baza):
    baza.drop(baza[baza['MISSING_DATA'] == True].index, inplace=True)
    baza.reset_index(drop=True, inplace=True)

usun_wiersze_z_brakami(baza)  #usuwam wszystkie wiersze gdzie missing data jest true

In [39]:
len(baza)

399997

In [40]:
baza.sort_values('TIMESTAMP',inplace = True)

In [41]:
baza.head()

,TRIP_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
577,1372636853620000380,1372636853,A,False,"[[-8.610291,41.140746],[-8.6103,41.140755],[-8..."
3,1372636854620000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
0,1372636858620000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
10,1372636875620000233,1372636875,A,False,"[[-8.619894,41.148009],[-8.620164,41.14773],[-..."
64,1372636896620000360,1372636896,A,False,"[[-8.617599,41.146137],[-8.617581,41.14593],[-..."


In [45]:
#zamiana timestampa na bardzije przyjazny zapis
baza['year'] = baza['TIMESTAMP'].apply(lambda x :datetime.datetime.fromtimestamp(x).year) 
baza['month'] = baza['TIMESTAMP'].apply(lambda x :datetime.datetime.fromtimestamp(x).month) 
baza['month_day'] = baza['TIMESTAMP'].apply(lambda x :datetime.datetime.fromtimestamp(x).day) 
baza['hour'] = baza['TIMESTAMP'].apply(lambda x :datetime.datetime.fromtimestamp(x).hour) 
baza['week_day'] = baza['TIMESTAMP'].apply(lambda x :datetime.datetime.fromtimestamp(x).weekday()) 

In [50]:
baza.head(5) #pierwsze 5 rekordów 


,TRIP_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,year,month,month_day,hour,week_day
577,1372636853620000380,1372636853,A,False,"[[-8.610291,41.140746],[-8.6103,41.140755],[-8...",2013,7,1,2,0
3,1372636854620000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",2013,7,1,2,0
0,1372636858620000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",2013,7,1,2,0
10,1372636875620000233,1372636875,A,False,"[[-8.619894,41.148009],[-8.620164,41.14773],[-...",2013,7,1,2,0
64,1372636896620000360,1372636896,A,False,"[[-8.617599,41.146137],[-8.617581,41.14593],[-...",2013,7,1,2,0
2,1372636951620000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",2013,7,1,2,0
51,1372636956620000167,1372636956,A,False,"[[-8.613297,41.15439],[-8.613306,41.153877],[-...",2013,7,1,2,0
5,1372636965620000231,1372636965,A,False,"[[-8.615502,41.140674],[-8.614854,41.140926],[...",2013,7,1,2,0
101,1372637072620000367,1372637072,A,False,"[[-8.617599,41.146254],[-8.617608,41.146074],[...",2013,7,1,2,0
308,1372637084620000285,1372637084,A,False,"[[-8.585685,41.148576],[-8.585766,41.148936],[...",2013,7,1,2,0


In [57]:
baza[-5:] # ostatnie 5 rekordów

,TRIP_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,year,month,month_day,hour,week_day
399986,1380278569620000260,1380278569,A,False,"[[-8.606331,41.144661],[-8.60688,41.144895],[-...",2013,9,27,12,4
399950,1380278662620000249,1380278662,A,False,"[[-8.640126,41.159718],[-8.64054,41.15979],[-8...",2013,9,27,12,4
399960,1380278748620000304,1380278748,A,False,"[[-8.613972,41.176368],[-8.613963,41.176386],[...",2013,9,27,12,4
399976,1380279405620000681,1380279405,A,False,"[[-8.690022,41.240403]]",2013,9,27,12,4
399991,1380279526620000681,1380279526,A,False,"[[-8.690112,41.240241],[-8.690175,41.240178],[...",2013,9,27,12,4
